In [1]:
# This notebook trains the nuclear-segmentation version of Mesmer
import os
import errno
import numpy as np 
import deepcell
from deepcell_toolbox.multiplex_utils import multiplex_preprocess

In [2]:
# create folder for this set of experiments
experiment_folder = "nuclear_model"
MODEL_DIR = os.path.join("/data/analyses", experiment_folder)
NPZ_DIR = "/data/npz_data/20201018_freeze/"
LOG_DIR = '/data/logs'

if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [ ]:
from deepcell.model_zoo.panopticnet import PanopticNet
from tensorflow.keras.optimizers import SGD, Adam
from deepcell.utils.train_utils import rate_scheduler
from deepcell import image_generators
from deepcell.utils import train_utils
from tensorflow.python.keras.losses import MSE
from deepcell import losses
from deepcell.utils.train_utils import get_callbacks
from deepcell.utils.train_utils import count_gpus

for seed in [1, 2, 3]:
    print("Training model for {}".format(seed))
    npz_name = "20201018_multiplex_seed_{}_nuclear_".format(seed)

    train_dict = np.load(NPZ_DIR + npz_name + "train_512x512.npz", allow_pickle=True)
    val_dict = np.load(NPZ_DIR + npz_name + "val_256x256.npz", allow_pickle=True)
    
    # initialize new model
    new_model = PanopticNet(
        backbone='resnet50',
        input_shape=(256, 256, 2),
        norm_method=None,
        num_semantic_heads=2,
        num_semantic_classes=[1, 3], # inner distance, pixelwise
        location=True,  # should always be true
        include_top=True,
        use_imagenet=True)
    
    
    X_train = train_dict['X']
    y_train = train_dict['y'][..., 1:]
    print("X_train shape is {}, y_train shape is {}".format(X_train.shape, y_train.shape))
    
    
    X_val = val_dict['X']
    y_val = val_dict['y'][..., 1:]
    print("X_val shape is {}, y_val shape is {}".format(X_val.shape, y_val.shape))
        
    # set up training parameters
    model_name = npz_name
    n_epoch = 100  # Number of training epochs

    optimizer = Adam(lr=1e-4, clipnorm=0.001)
    lr_sched = rate_scheduler(lr=1e-4, decay=0.99)

    batch_size = 8

    min_objects = 0  # throw out images with fewer than this many objects
    seed=0
    print("Model name is {}".format(model_name))
    
    # create augmented dataset
    datagen = image_generators.CroppingDataGenerator(
        rotation_range=180,
        shear_range=0,
        zoom_range=(0.7, 1/0.7),
        horizontal_flip=True,
        vertical_flip=True,
        crop_size=(256, 256),
        float_dtype='float16',
        int_dtype='int16')

    datagen_val = image_generators.SemanticDataGenerator(
        rotation_range=0,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=0,
        vertical_flip=0,
        float_dtype='float16',
        int_dtype='int16')

    train_data = datagen.flow(
        {'X': X_train, 'y': y_train},
        seed=seed,
        transforms=['inner-distance', 'pixelwise'],
        transforms_kwargs={'pixelwise':{'dilation_radius': 1}, 
                          'inner-distance': {'erosion_width': 1, 'alpha': 'auto'}},
        min_objects=min_objects,
        batch_size=batch_size)

    val_data = datagen_val.flow(
        {'X': X_val, 'y': y_val},
        seed=seed,
        transforms=['inner-distance', 'pixelwise'],
        transforms_kwargs={'pixelwise':{'dilation_radius': 1},
                          'inner-distance': {'erosion_width': 1, 'alpha': 'auto'}},
        min_objects=min_objects,
        batch_size=batch_size)
    
    print('generators created')
    
    # set up losses
    def semantic_loss(n_classes):
        def _semantic_loss(y_pred, y_true):
            if n_classes > 1:
                return 0.01 * losses.weighted_categorical_crossentropy(
                    y_pred, y_true, n_classes=n_classes)
            return MSE(y_pred, y_true)
        return _semantic_loss


    loss = {}

    # Give losses for all of the semantic heads
    for layer in new_model.layers:
        if layer.name.startswith('semantic_'):
            n_classes = layer.output_shape[-1]
            loss[layer.name] = semantic_loss(n_classes)
            
    # compile model
    new_model.compile(loss=loss, optimizer=optimizer)
    
    # train model
    model_path = os.path.join(MODEL_DIR, '{}.h5'.format(model_name))
    loss_path = os.path.join(MODEL_DIR, '{}.npz'.format(model_name))

    num_gpus = count_gpus()

    print('Training on', num_gpus, 'GPUs.')

    train_callbacks = get_callbacks(
        model_path,
        lr_sched=lr_sched,
        #tensorboard_log_dir=LOG_DIR,
        save_weights_only=num_gpus >= 2,
        monitor='val_loss',
        verbose=1)

    loss_history = new_model.fit_generator(
        train_data,
        steps_per_epoch=333,
        epochs=n_epoch,
        validation_data=val_data,
        validation_steps=val_data.y.shape[0] // batch_size,
        callbacks=train_callbacks)


Training model for 1


W1106 05:17:27.223625 140580942575424 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


X_train shape is (2664, 512, 512, 2), y_train shape is (2664, 512, 512, 1)


W1106 05:19:07.941479 140580942575424 semantic.py:111] X data dtype is float32: this will increase memory use during preprocessing. Consider using a smaller dtype
W1106 05:19:07.943075 140580942575424 semantic.py:115] y data dtype is int32: this will increase memory use during preprocessing. Consider using a smaller dtype.


X_val shape is (3224, 256, 256, 2), y_val shape is (3224, 256, 256, 1)
Model name is 20201018_multiplex_seed_1_nuclear_


W1106 05:20:31.702312 140580942575424 semantic.py:111] X data dtype is float32: this will increase memory use during preprocessing. Consider using a smaller dtype
W1106 05:20:31.703572 140580942575424 semantic.py:115] y data dtype is int32: this will increase memory use during preprocessing. Consider using a smaller dtype.


generators created
Training on 1 GPUs.
Epoch 1/100


W1106 05:20:58.029047 140580942575424 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


332/333 [============================>.] - ETA: 1s - loss: 0.0214 - semantic_0_loss: 0.0153 - semantic_1_loss: 0.0060
Epoch 00001: val_loss improved from inf to 0.08962, saving model to /data/analyses/nuclear_model/20201018_multiplex_seed_1_nuclear_.h5
333/333 [==============================] - 570s 2s/step - loss: 0.0213 - semantic_0_loss: 0.0153 - semantic_1_loss: 0.0060 - val_loss: 0.0896 - val_semantic_0_loss: 0.0354 - val_semantic_1_loss: 0.0542
Epoch 2/100
332/333 [============================>.] - ETA: 1s - loss: 0.0132 - semantic_0_loss: 0.0089 - semantic_1_loss: 0.0042
Epoch 00002: val_loss improved from 0.08962 to 0.05563, saving model to /data/analyses/nuclear_model/20201018_multiplex_seed_1_nuclear_.h5
333/333 [==============================] - 494s 1s/step - loss: 0.0132 - semantic_0_loss: 0.0089 - semantic_1_loss: 0.0042 - val_loss: 0.0556 - val_semantic_0_loss: 0.0326 - val_semantic_1_loss: 0.0230
Epoch 3/100
332/333 [============================>.] - ETA: 1s - loss: 0.0